In [1]:
import tensorflow as tf
from tensorflow.contrib.rnn import RNNCell
from tensorflow.python.ops import variable_scope as vs
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from func.jupyter_tensorboard import show_graph

from func.midi_to_statematrix import *
from func.data import *
import func.multi_training



import os
#import cPickle as pickle
import pickle
import signal
import numpy as np
import random
import pygame

path = os.getcwd()

In [2]:
def play_music(music_file):
    """
    stream music with mixer.music module in blocking manner
    this will stream the sound from disk while playing
    """
    clock = pygame.time.Clock()
    try:
        pygame.mixer.music.load(music_file)
        print("Music file %s loaded!" % music_file)
    except pygame.error:
        print("File %s not found! (%s)" % (music_file, pygame.get_error()))
        return
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        # check if playback has finished
        clock.tick(30)

freq = 44100    # audio CD quality
bitsize = -16   # unsigned 16 bit
channels = 2    # 1 is mono, 2 is stereo
buffer = 1024    # number of samples
pygame.mixer.init(freq, bitsize, channels, buffer)
# optional volume 0 to 1.0
pygame.mixer.music.set_volume(0.8)


In [ ]:
play_music(path + '/music_test/' + 'beethoven_opus10_1.mid')

In [3]:
pcs = func.multi_training.loadPieces(path  + '/music_test')

Loaded alb_esp2
Loaded alb_esp5
Loaded appass_2
Loaded appass_3
Loaded bach_846
Loaded bach_847
Loaded bach_850
Loaded beethoven_hammerklavier_1
Loaded beethoven_les_adieux_1
Loaded beethoven_les_adieux_2
Loaded beethoven_opus10_2
Loaded beethoven_opus10_3
Loaded beethoven_opus22_1
Loaded beethoven_opus22_4
Loaded beethoven_opus90_2


In [5]:
batch_width = 10 # number of sequences in a batch
batch_len = 16*8 # length of each sequence
division_len = 16 # interval between possible start locations

def loadPieces(dirpath):

    pieces = {}

    for fname in os.listdir(dirpath):
        if fname[-4:] not in ('.mid','.MID'):
            continue

        name = fname[:-4]

        outMatrix = midiToNoteStateMatrix(os.path.join(dirpath, fname))
        if len(outMatrix) < batch_len:
            continue

        pieces[name] = outMatrix
        print("Loaded {}".format(name))

    return pieces

def getPieceSegment(pieces):
    pcs=pieces.values()
    piece_output = random.choice(list(pcs))
    start = random.randrange(0,len(piece_output)-batch_len,division_len)
    
    # print "Range is {} {} {} -> {}".format(0,len(piece_output)-batch_len,division_len, start)

    seg_out = piece_output[start:start+batch_len]
    seg_in = noteStateMatrixToInputForm(seg_out)

    return seg_in, seg_out

def getPieceBatch(pieces):
    i,o = zip(*[getPieceSegment(pieces) for _ in range(batch_width)])
    return numpy.array(i), numpy.array(o)

In [6]:
def trainPiece(model,pieces,epochs,start=0):
    stopflag = [False]
    def signal_handler(signame, sf):
        stopflag[0] = True
    old_handler = signal.signal(signal.SIGINT, signal_handler)
    for i in range(start,start+epochs):
        if stopflag[0]:
            break
        error = model.update_fun(*getPieceBatch(pieces))
        if i % 100 == 0:
            print("epoch {}, error={}".format(i,error))
        if i % 500 == 0 or (i % 100 == 0 and i < 1000):
            xIpt, xOpt = map(numpy.array, getPieceSegment(pieces))
            noteStateMatrixToMidi(numpy.concatenate((numpy.expand_dims(xOpt[0], 0), model.predict_fun(batch_len, 1, xIpt[0])), axis=0),'output/sample{}'.format(i))
            pickle.dump(model.learned_config,open('output/params{}.p'.format(i), 'wb'))
    signal.signal(signal.SIGINT, old_handler)

In [7]:
song={}
song['beethoven_hammerklavier_1']=pcs['beethoven_hammerklavier_1']

In [170]:
def Model(t_layer_sizes,p_layer_sizes,xs,ys):

    t_input_size = 80
    lstm_time=[]
    for i in t_layer_sizes:
        lstm_time.append(tf.contrib.rnn.LSTMCell(i))

    time_model=tf.contrib.rnn.MultiRNNCell(lstm_time)        
    init_state_time=time_model.zero_state(tf.shape(ys)[0],tf.float32)
    
    
    
    #output, self.final_state = tf.nn.dynamic_rnn(...)


    
    
    
    with tf.variable_scope('lstm1'):
        #tf.get_variable
        outputs_time,final_state_time=tf.nn.dynamic_rnn(time_model, xs, initial_state = init_state_time, dtype = tf.float32)
        for one_lstm_cell in lstm_time:
            one_kernel, one_bias = one_lstm_cell.variables
            # I think TensorBoard handles summaries with the same name fine.
            tf.summary.histogram("Kernel-time", one_kernel)
            tf.summary.histogram("Bias-time", one_bias)    
    p_input_size = t_layer_sizes[-1] + 2


    lstm_pitch=[]

    for i in p_layer_sizes:
        lstm_pitch.append(tf.contrib.rnn.LSTMCell(i))
    lstm_pitch.append(tf.contrib.rnn.LSTMCell(2))


    pitch_model=tf.contrib.rnn.MultiRNNCell(lstm_pitch)

    init_state_pitch=pitch_model.zero_state(tf.shape(ys)[0],tf.float32)
    with tf.variable_scope('lstm2'):
        outputs_pitch,final_state_pitch=tf.nn.dynamic_rnn(pitch_model,outputs_time,initial_state = init_state_pitch,dtype = tf.float32)
        for one_lstm_cell in lstm_pitch:
            one_kernel, one_bias = one_lstm_cell.variables
            # I think TensorBoard handles summaries with the same name fine.
            tf.summary.histogram("Kernel-pitch", one_kernel)
            tf.summary.histogram("Bias-pitch", one_bias)    
        #variables_names =[v.name for v in tf.trainable_variables()]
        #values = sess.run(variables_names)
                
        #for k,v in zip(variables_names, values):
            #tf.summary.histogram('parameters/{}'.format(k),v)
                    
    return outputs_pitch


def cross_entropy(output, input_y):
    with tf.name_scope('cross_entropy'):
        
        ce = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=input_y, logits=output))

    return ce


def train_step(loss, learning_rate=1e-3):
    with tf.name_scope('train_step'):
        step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    return step


def evaluate(output, input_y):
    with tf.name_scope('evaluate'):
        pred = tf.argmax(output, axis=1)
        error_num = tf.count_nonzero(pred - tf.cast(input_y, tf.int64), name='error_num')
        #tf.summary.scalar('LeNet_error_num', error_num)
    return error_num

def training(song,t_layer_sizes,p_layer_sizes, pre_trained_model=None):
    
    tf.reset_default_graph()
    # define the variables and parameter needed during training
    with tf.name_scope('inputs'):
        xs = tf.placeholder(tf.float32, [None,None, t_input_size])
        ys = tf.placeholder(tf.float32, [None,None, 2])
        
        
    output= Model(t_layer_sizes,p_layer_sizes,xs,ys)
    print(output.shape)
    loss=cross_entropy(output, ys)
    
    iters = int(np.array(list(song.values())[0]).shape[0] / batch_len)
    print('number of batches for training: {}'.format(iters))

    step = train_step(loss)
    eve = evaluate(output, ys)

    iter_total = 0
    best_acc = 0
    cur_model_name = 'amadeus'

    epoch=1
    
    with tf.Session() as sess:
        
        merge = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter("log/{}".format(cur_model_name), sess.graph)
        saver = tf.train.Saver()
        
        sess.run(tf.global_variables_initializer())
        #train_writer = tf.summary.FileWriter("log/", graph=tf.get_default_graph())

        # try to restore the pre_trained
        if pre_trained_model is not None:
            try:
                print("Load the model from: {}".format(pre_trained_model))
                saver.restore(sess, 'model_save/{}'.format(pre_trained_model))
            except Exception:
                print("Load model Failed!")
                pass

        for epc in range(epoch):
            print("epoch {} ".format(epc + 1))

            for itr in range(iters):
                
                training_batch_x,training_batch_y= map(numpy.array, getPieceSegment(song))
                _, cur_loss = sess.run([step, loss], feed_dict={xs: training_batch_x, ys: training_batch_y})
                #out=sess.run(output, feed_dict={xs: training_batch_x, ys: training_batch_y})
                
                #variables_names =[v.name for v in tf.trainable_variables()]
                #print(variables_names)
                    
                merge_result=sess.run(merge, feed_dict={xs: training_batch_x, ys: training_batch_y})
                
                
                
                
                saver.save(sess,'model_save/{}'.format(cur_model_name))
                
                
                
                train_writer.add_summary(merge_result,itr)
               
                
                print(cur_loss)
                if itr==iters-1:
                    out=sess.run(output, feed_dict={xs: training_batch_x, ys: training_batch_y})
                    print(out)
                    u=notenoteStateMatrixToMidi(out, name="example")
                    print(u)
                    
                    
                


In [171]:
t_input_size=80

In [172]:
a=training(song,[300,300],[100,50], pre_trained_model=None)

(?, ?, 2)
number of batches for training: 33
epoch 1 
0.707101
0.704633
0.700442
0.696929
0.694408
0.689982
0.688191
0.684892
0.683582
0.680563
0.677973
0.675444
0.673013
0.670413
0.66738
0.66517
0.662073
0.660162
0.657423
0.653936
0.651794
0.649536
0.648479
0.644155
0.642763
0.639464
0.638895
0.63758
0.63437
0.631714
0.631496
0.631399
0.626523
[[[-0.00187975 -0.00132048]
  [-0.00339546 -0.00234873]
  [-0.00447675 -0.00314189]
  ..., 
  [-0.31019363 -0.16779122]
  [-0.31150779 -0.16702887]
  [-0.31272268 -0.16627012]]

 [[-0.0019235  -0.00136462]
  [-0.00360263 -0.00254688]
  [-0.00505324 -0.0036632 ]
  ..., 
  [-0.31740251 -0.1604653 ]
  [-0.31842807 -0.1597805 ]
  [-0.31936499 -0.15911718]]

 [[-0.0019098  -0.00132471]
  [-0.0035402  -0.0023636 ]
  [-0.00488256 -0.00317056]
  ..., 
  [-0.31935802 -0.16782653]
  [-0.3203443  -0.1670316 ]
  [-0.32125655 -0.16625173]]

 ..., 
 [[-0.00190149 -0.00140312]
  [-0.00350693 -0.00272268]
  [-0.00484403 -0.00409525]
  ..., 
  [-0.31257224 -0.15

NameError: name 'notenoteStateMatrixToMidi' is not defined

In [148]:
# show the graph
from func.jupyter_tensorboard import show_graph 
tf.reset_default_graph()
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('model_save/amadeus.meta')
    graph = tf.get_default_graph()
    show_graph(graph)

In [159]:
out

NameError: name 'out' is not defined